Sometimes it's impossible to model the dependent variable Y as a linear equation of the independent variable X. We could model the dependent variable Y to be a high order polynomial equation of the dependent variable X, thus converting the problem into the standard form of linear regressions.

It can also be seing that calculating the weights or coefficients of all the terms in a polynomial function using gradiend descent has a time complexity of O(n^2), where n is the numer of features in the training data. Similarly, the algorithmic complexity of calculating the coefficients of all the terms in a third-order polynomial equation is O(n^3). It's apparent that the time complexity of gradient descent increases geomtrically with the number of features of the model. Thus, gradient descent on its own is not efficient enough yo model nonlinear regression models with a large number of features.

ANNs, on the other hand, are very efficient at modeling nonlinear regression models of data with a high number of features. ANNs are modeled form the behavior of the central nervous system of organisms. The brain processes information and generates electric signals that are transported thorugh the network of neural fibers to the various organs of the organism. **The actual processing of sensoty signals, however, is performed by several complex combinations of these neurons**. Of course, each neuron is capable of preocessing an exremely small portion og the information preocessed by the brain.

The dendrites are used to receive signals from other neuros and can be thought of as the input to the neuron. Similarly, the axon the the neuron is analougous to the output of the neuron. The neuron can thus be mathematically represented as a function that processes several inputs and produces a single output. The interconnecting space between two neurons is called a **synapse**.

**Perceptron** can be used on its own and is effective enough to estimate supervised machine learning models such as **linear regression** and **logistic regression**. However, complex nonlinear data can be better modeled with several interconnected perceptrons.

ANNs can be braoadly classified into **feed-forward neural networks** and **recurrent neural networks**. The difference between these two types of ANNs is that in feed-forward neural networks, the conections between nodes ot the ANN do not form a directed cycle as opposed yo recurrent neural networks where the node interconnections do form a directed cycle. Thus, in feed-forward neural networks, each node in a given layer of the ANN receives input only from the nodes in the immediate previous layer of the ANN.

### Multilayer Perceptron

This model represents a basic feed-forward neural network and is versatile enough to model regression and classificaiton problem in the domain of supervised learning. All the inputs flow through a feed-forward neural network in a single direction. This is direct consequence of the fact that **there is no feddback** from or to any layer in a feed-forward neural network.

Usin a single layer of perceptrons would mea using only a single activation function, which is the equivalent to using **logistic regression** to model the training data. This would mean that the model cannot fit nonlinear data, which is the primary innovation of ANNs.

We can use a multilayer peceptron to perform a **binary or multiclass classifications**. A multilayer perceptron ANN can be trained using the **backpropagation algorithm**.